# Solving stochastic control problems using python

In this series of work, we would like to solve a class of control problems 
driven by Stochastic Differential Equation (SDE) 
using both the analytical technique and the numerical technique.
The code is implemented and illustrated by python.

For this, we introduce the class of stochastic control problems as follows
$$ V(t,x) := \inf\limits_{u} J(t,x,u) := \inf\limits_{u} \mathbb{E} \left[g(X^{t,x,u}_T) + \int_t^T f(t,X^{t,x,u}_s, u_s) ds \right],$$
where $X^{t,x,u}$ satisfies the following SDE
$$ X^{t,x,u}_s = x + \int_t^s b(r, X^{t,x,u}_r, u_r) dr + \int_t^s \sigma(r, X^{t,x,u}_r, u_r) dW_r,$$
where $T > 0$, $(t,x) \in [0,T] \times \mathbb{R}$, $b$, $\sigma$, $g$, $f$ are deterministic functions with sufficient regularities, $u$ is a square-integrable control process, and $W$ is a standard Brownian motion.

Here, $X^{t,x,u}$ is commonly called the controlled process because it is controlled by the control process $u$ through the third variable in $b$ and $\sigma$.
$J$ is the cost function given the initial time $t$ and initial value $x$ while $g$ and $h$ are the terminal costs and running costs respectively.
In a stochastic control problem, we aim at choosing the optimal control process $u$ such that the cost function $J$ is minimized.
Then, we denote the minimized $J$ by the value function $V$.

In the section [HJB equation](#HJB-equation), we aim to heuristically show the equivalence between the value function and the Hamilton–Jacobi–Bellman (HJB) equation. 
Subsequently in the section [Merton problem](#Merton-problem), we apply this HJB equation technique to solve the Merton consumption and portfolio problem. 
Lastly in the section [Monte Carlo simulation](#Monte-Carlo-simulation), we implement the Monte Carlo simulation using python to check the formula derived in the section [Merton problem](#Merton-problem).

## HJB equation

For any fixed $u$, we note the flow property $X^{s,X^{t,x,u}_s,u}_r = X^{t,x,u}_r$ holds for $0 \le t \le s \le r \le T$.
The idea here is that the SDE that starts at $s$ with initial value $X^{t,x,u}_s$ is equivalent to the SDE that starts at $t$ with initial value $x$.
Then we see that
\begin{align}
J(t,x,u)    &= \mathbb{E} \left[ g(X^{s,X^{t,x,u}_s,u}_T) + \int_s^T f(r, X^{s,X^{t,x,u}_s,u}_r, u_r) dr + \int_t^s f(r, X^{t,x,u}_r, u_r) dr \right] \\\\
            &= \mathbb{E} \left[J(s, X^{t,x,u}_s, u) + \int_t^s f(r, X^{t,x,u}_r, u_r) dr \right].
\end{align}
Taking infimum on both sides yields the famous Bellman's principle of optimality
\begin{equation}
    \label{eq1} \tag{1}
    V(t,x) = \inf\limits_u \mathbb{E} \left[V(s, X^{t,x,u}_s) + \int_t^s f(r, X^{t,x,u}_r, u_r) dr \right].
\end{equation}
In the sequel, we denote $V_t$ and $V_x$ the partial derivatives of $V$ with respect to the first and second variable respectively.
By putting the following Itô's lemma 
$$ \mathbb{E} \left[ V(s, X^{t,x,u}_s) \right] = V(t,x) + \mathbb{E} \left[ \int_t^s \left( V_t(r,X^{t,x,u}_r) + b(r,X^{t,x,u}_r, u_r)V_x(r,X^{t,x,u}_r) + \frac{1}{2}\sigma(r,X^{t,x,u}_r, u_r)V_{xx}(r,X^{t,x,u}_r) \right) dr \right]$$
into the Bellman's principle of optimality \eqref{eq1} and dividing both sides of equality by $s-t$, we obtain
$$ 0 = \inf\limits_u \mathbb{E} \left[\frac{1}{s-t} \int_t^s \left(V_t(r,X^{t,x,u}_r) + b(r,X^{t,x,u}_r, u_r)V_x(r,X^{t,x,u}_r) + \frac{1}{2}\sigma(r,X^{t,x,u}_r, u_r)V_{xx}(r,X^{t,x,u}_r) + f(r, X^{t,x,u}_r, u_r) \right) dr \right]. $$
Finally, pass the limit $(s - t) \rightarrow 0$ to get the HJB equation
$$ 0 = V_t(t, x) + \inf\limits_u \left(b(t, x, u)V_x(t, x) + \frac{1}{2}\sigma(t, x, u)V_{xx}(t, x) + f(t, x, u) \right), $$
with the terminal condition
$$ V(T,x) = g(x).$$

<br>
We stress that the derivation above is not fully rigorous.
The fully rigorous version is too long to be presented here and can be found in Chapter 4 of (yong zhou.....).
It involves the epsilon definition of infimum and the weak formulation of the control problem,
where the probability space is also controlled.

## Merton problem

Assume that the stock process $S^1$ follows the following SDE
$$ S^1_t = S^1_0 + \alpha \int_0^t S^1_s ds + \sigma \int_0^t S^1_s dW_s, $$
where $\alpha \in \mathbb{R}$ is the return rate and $\sigma > 0$ is the volatility of the stock.
Suppose that we can continuously choose the portion $u_t$ we would like to invest in the stock $S^1$ while putting the rest of the portion $1 - u_t$ in the bank $S^0$,
earning the return rate of $r \in \mathbb{R}$ with zero volatility, i.e.
$$ S^0_t = S^0_0 + r \int_0^t S^0_s ds. $$
Note that $r \le \alpha$, else we should just put everything in the bank.
Here we face a trade-off in the sense that investing more in the stock yields a higher return but a more risky portfolio.

Denote by $X^{0,x_0,u}$ the wealth process using the strategy $u$ with initial wealth $x_0 > 0$.
Since $S^1_t = u_t X^{0,x_0,u}_t$ and $S^0_t = ( 1- u_t ) X^{0,x_0,u}_t$, we have
\begin{align*}
X^{0,x_0,u}_t   &= x_0 + \alpha \int_0^t u_s X^{0,x_0,u}_s ds + \sigma \int_0^t u_sX^{0,x_0,u}_s dW_s + r \int_0^t (1 - u_s) X^{0,x_0,u}_s ds \\
                &= x_0 + \int_0^t \left( r + (\alpha - r) u_s \right) X^{0,x_0,u}_s ds + \int_0^t \sigma u_sX^{0,x_0,u}_s dW_s,
\end{align*}
and we aim to minimize the following
$$ -\mathbb{E} \left[ \frac{(X^{0,x_0,u}_T)^{1-\gamma}}{1-\gamma} \right]. $$
Comparing the functions with the notations used in the section [HJB equation](#HJB-equation) yields $b(t,x,u) = (r + (\alpha - r)u)x$, $\sigma(t,x,u) = \sigma u x$, $f(t,x,u) = 0$, $g(x) = -\frac{x^{1-\gamma}}{1-\gamma}$.










## Monte Carlo simulation

## Appendix

For any fixed $u$, we first see how the flow property $X^{s,X^{t,x,u}_s,u}_r = X^{t,x,u}_r$ holds for $0 \le t \le s \le r \le T$.
Indeed, by introducing the notation $Z_r = X^{s,X^{t,x,u}_s,u}_r$ we obtain
\begin{align}
    Z_r &= X^{t,x,u}_s + \int_s^r b(k, Z_k, u_k) dk + \int_s^r \sigma(k, Z_k, u_k) dW_k \\\\
        &= x + \int_t^s b(k, X^{t,x,u}_k, u_k) dk + \int_s^r \sigma(k, X^{t,x,u}_k, u_k) dW_k + \int_s^r b(k, Z_k, u_k) dk + \int_s^r \sigma(k, Z_k, u_k) dW_k.
\end{align}
Since we similarly also have
$$ X^{t,x,u}_r = x + \int_t^s b(k, X^{t,x,u}_k, u_k) dk + \int_t^s \sigma(k, X^{t,x,u}_k, u_k) dW_k + \int_s^r b(k, X^{t,x,u}_k, u_k) dk + \int_s^r \sigma(k, X^{t,x,u}_k, u_k) dW_k,$$
we can conclude the flow property $X^{s,X^{t,x,u}_s,u}_r = Z_r = X^{t,x,u}_r$.

<h2 id="importing_libraries">Importing required libraries</h2>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Though Linear regression is very good to solve many problems, it cannot be used for all datasets. First recall how linear regression, could model a dataset. It models a linear relation between a dependent variable y and independent variable x. It had a simple equation, of degree 1, for example y = $2x$ + 3.

In [ ]:
x = np.arange(-5.0, 5.0, 0.1)

##You can adjust the slope and intercept to verify the changes in the graph
y = 2*(x) + 3
y_noise = 2 * np.random.normal(size=x.size)
ydata = y + y_noise
#plt.figure(figsize=(8,6))
plt.plot(x, ydata,  'bo')
plt.plot(x,y, 'r') 
plt.ylabel('Dependent Variable')
plt.xlabel('Indepdendent Variable')
plt.show()

Non-linear regressions are a relationship between independent variables $x$ and a dependent variable $y$ which result in a non-linear function modeled data. Essentially any relationship that is not linear can be termed as non-linear, and is usually represented by the polynomial of $k$ degrees (maximum power of $x$). 

$$ \ y = a x^3 + b x^2 + c x + d \ $$

Non-linear functions can have elements like exponentials, logarithms, fractions, and others. For example: $$ y = \log(x)$$
    
Or even, more complicated such as :
$$ y = \log(a x^3 + b x^2 + c x + d)$$

[1] J. Yong and X.Y. Zhou. Stochastic controls: Hamiltonian systems and HJB equations, volume 43. Springer, 1999.